# Using RiskGPT in Google Colab

This notebook demonstrates how to use RiskGPT in Google Colab by installing it directly from the GitHub repository and setting up the OpenAI API key.


## 1. Install RiskGPT from GitHub

Since RiskGPT is not available on PyPI, we'll install it directly from the GitHub repository.


In [ ]:
# Install Python 3.12 in Colab (if needed)
!apt-get update
!apt-get install python3.12 python3.12-dev python3.12-distutils -y
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1

# Verify Python version
!python3 --version

In [ ]:
# Install pip for Python 3.12
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.12

# Install RiskGPT directly from GitHub
!pip install git+https://github.com/thwolter/riskgpt.git

# Alternative: If you have access to the wheel file
# !pip install /path/to/dist/riskgpt-0.1.0-py3-none-any.whl

## 2. Set up the OpenAI API Key

You need to set your OpenAI API key to use RiskGPT. In Colab, you can set environment variables directly in the notebook.


In [ ]:
import os
from getpass import getpass

# Prompt for OpenAI API key (this way it won't be visible in the notebook)
openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Optional: Set other configuration variables if needed
os.environ["OPENAI_MODEL_NAME"] = "openai:gpt-4.1-nano"  # Default model
os.environ["MEMORY_TYPE"] = "buffer"  # Default memory type

## 3. Basic Usage Example

Here's a simple example of how to use RiskGPT for risk analysis.


In [ ]:
# Import required modules
import asyncio
from riskgpt.logger import configure_logging
from riskgpt.models.common import BusinessContext
from riskgpt.models.chains.risk import Risk
from riskgpt.models.enums import AudienceEnum
from riskgpt.models.chains.questions import (
    ChallengeQuestionsRequest,
    ChallengeRiskRequest,
)
from riskgpt.chains.challenge_questions import challenge_questions_chain
from riskgpt.chains.challenge_risk import challenge_risk_chain
from riskgpt.models.workflows.context import ResearchRequest
from riskgpt.workflows.research import research

# Configure logging
configure_logging()

# Create a business context
context = BusinessContext(
    project_id="CLOUD-2023",
    project_description="Migrate on-premises infrastructure to cloud services",
    domain_knowledge="The company is a financial services provider with strict regulatory requirements",
    business_area="IT Infrastructure",
    industry_sector="Financial Services",
)

# Create a sample risk
sample_risk = Risk(
    title="Data Security Breach",
    description="Risk of unauthorized access to sensitive data during migration",
    category="Security",
)


# Define an async function to run the challenge_questions chain
async def run_challenge_questions():
    # Create a request
    questions_request = ChallengeQuestionsRequest(
        business_context=context,
        audience=AudienceEnum.risk_internal,
        focus_areas=["data security", "compliance", "service continuity"],
        num_questions=5,
    )

    # Run the chain
    response = await challenge_questions_chain(questions_request)
    print(f"Generated {len(response.questions)} challenging questions:")
    for i, question in enumerate(response.questions, 1):
        print(f"{i}. {question}")
    return response


# Define an async function to run the challenge_risk chain
async def run_challenge_risk():
    # Create a request
    risk_request = ChallengeRiskRequest(
        risk=sample_risk,
        business_context=context,
        audience=AudienceEnum.risk_internal,
        focus_areas=["data encryption", "access controls", "regulatory compliance"],
        num_questions=5,
    )

    # Run the chain
    response = await challenge_risk_chain(risk_request)
    print(
        f"Generated {len(response.questions)} challenging questions for risk '{sample_risk.title}':"
    )
    for i, question in enumerate(response.questions, 1):
        print(f"{i}. {question}")
    return response


# Define an async function to run the research workflow
async def run_enrich_context():
    # Create a request
    enrich_request = ResearchRequest(
        business_context=context,
        focus_keywords=["cloud migration", "financial services", "data security"],
        time_horizon_months=12,
    )

    # Run the workflow
    response = await research(enrich_request)
    print("Sector Summary:")
    print(response.summary)
    print("\nWorkshop Recommendations:")
    for i, rec in enumerate(response.recommendations, 1):
        print(f"{i}. {rec}")
    if response.full_report:
        print("\nFull Report:")
        print(
            response.full_report[:500] + "..."
            if len(response.full_report) > 500
            else response.full_report
        )
    return response


# Execute the async functions
async def run_all():
    print("Running challenge_questions chain...")
    await run_challenge_questions()

    print("\nRunning challenge_risk chain...")
    await run_challenge_risk()

    print("\nRunning research workflow...")
    await run_enrich_context()


# Run all the examples
asyncio.run(run_all())

## 4. Combining the Workflows

Now let's see how we can combine these workflows to create a more comprehensive risk analysis.


In [ ]:
async def combined_workflow():
    # Step 1: Enrich the context with external information
    print("Step 1: Enriching context with external information...")
    enrich_response = await research(
        ResearchRequest(
            business_context=context,
            focus_keywords=["cloud migration", "financial services", "data security"],
            time_horizon_months=12,
        )
    )

    # Step 2: Generate challenging questions based on the enriched context
    print("\nStep 2: Generating challenging questions...")
    questions_response = await challenge_questions_chain(
        ChallengeQuestionsRequest(
            business_context=context,
            audience=AudienceEnum.risk_internal,
            focus_areas=["data security", "compliance", "service continuity"],
            num_questions=5,
        )
    )

    # Step 3: Generate challenging questions for a specific risk
    print("\nStep 3: Generating challenging questions for a specific risk...")
    risk_response = await challenge_risk_chain(
        ChallengeRiskRequest(
            risk=sample_risk,
            business_context=context,
            audience=AudienceEnum.risk_internal,
            focus_areas=["data encryption", "access controls", "regulatory compliance"],
            num_questions=5,
        )
    )

    # Return all responses
    return {
        "enrich_response": enrich_response,
        "questions_response": questions_response,
        "risk_response": risk_response,
    }


# Execute the combined workflow
combined_results = asyncio.run(combined_workflow())

# Display a summary of the results
print("\nSummary of Combined Workflow:")
print(
    f"- Enriched context with {len(combined_results['enrich_response'].recommendations)} recommendations"
)
print(
    f"- Generated {len(combined_results['questions_response'].questions)} challenging questions from business context"
)
print(
    f"- Generated {len(combined_results['risk_response'].questions)} challenging questions for risk '{sample_risk.title}'"
)

## 5. Troubleshooting

If you encounter any issues, here are some common troubleshooting steps:

1. **Python Version**: RiskGPT requires Python 3.12 or higher. Make sure you've installed and are using the correct Python version.

2. **API Key**: Ensure your OpenAI API key is valid and has sufficient credits.

3. **Dependencies**: If you encounter dependency issues, you can try installing them manually:

```python
!pip install langchain>=0.3.26 langchain-openai>=0.3.25 langgraph>=0.4.8 pydantic>=2.11.7
```

4. **Import Errors**: If you see import errors, make sure the package was installed correctly:

```python
!pip list | grep riskgpt
```
